In [1]:
from whoosh.index import create_in
from whoosh.fields import *

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

schema = Schema(title_ix=TEXT(stored=True), content_ix=TEXT, headers_ix=TEXT, url_ix=TEXT(stored=True))

ix = create_in("indexdir", schema)
writer = ix.writer()

prefix = 'https://vm009.rz.uos.de/crawl/'
start_url = prefix+'index.html'
agenda = [start_url]
visited_recently = []
headers_v = []


while agenda:
    url = agenda.pop()
    if url not in visited_recently:
        visited_recently.append(url)
        r = requests.get(url)

        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')

            for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                headers_v.append(header.get_text())

            content = soup.get_text()
            writer.add_document(title_ix = soup.title.get_text(strip=True) if soup.title else "No Title", content_ix=soup.get_text(), headers_ix=' '.join(headers_v), url_ix=url)
            
            headers_v = []
            
            
            for link in soup.find_all('a'):
                
                href = link.get('href')
                # if the href is a relative link, we need to join it with the base url. If the href is an absolut link the function urljoin will return the href as it is.
                full_url = urljoin(url, href)

                # we dont want to scrape other weppages than the ones that start with the prefix
                if full_url.startswith(prefix):
                        agenda.append(full_url)
writer.commit()


In [19]:
from whoosh.qparser import QueryParser
with ix.searcher() as searcher:
    # find entries with the words 'first' AND 'last'
    query = QueryParser("content_ix", ix.schema).parse("pixels")
    results = searcher.search(query)

    # print all results
    for r in results:
        print(r)

<Hit {'title_ix': 'Page 6', 'url_ix': 'https://vm009.rz.uos.de/crawl/page6.html'}>


In [13]:
uni_start = 'https://www.uni-osnabrueck.de/'
r = requests.get('https://vm009.rz.uos.de/crawl/page6.html')
soup = BeautifulSoup(r.content, 'html.parser')
print("TITEL: ",soup.title.get_text())
for text in soup.find_all('p'):
    print(text.get_text())
print('--------------------------------------')
print(soup.get_text())

TITEL:  Page 6
Author: ChatGPT-4
--------------------------------------



Page 6


This is Page 6

    In a world of pixels and bytes so torn,
Lived a quirky platypus and a geeky unicorn.
Platypus coded, with fins so deft,
While unicorn debugged, with every breath left.

"Did you know," said Plat, with a glint in his eye,
"That I'm a mammal that lays eggs, oh my!"
Unicorn chuckled, "And I've a horn of might,
But can't run basic scripts, try as I might."

Together they ventured, in realms of code and lore,
Fixing glitches, and chasing bugs galore.
For in this nerdy tale, so uniquely spun,
A platypus and unicorn made coding fun!
    
Author: ChatGPT-4
Page 2
Page 3
Page 7
Home Page



